In [7]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize

In [8]:
#Define some parameters for training
batch_size = 128
num_classes = 10
epochs = 30
num_iterators = 50

In [9]:
#Load in the data (If you want f)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Process
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

170500096/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
#Load in the data (If you want undersampled dataset)
undersample_path = '/home/ec2-user/SageMaker/efs/Data/UnderSamples/'
(_,_), (x_test, y_test) = cifar10.load_data()
x_train = np.load(undersample_path + 'X9Prop0.1.npy')
x_train = x_train.astype('float32')
x_train /= 255
x_test = x_test.astype('float32')
x_test /= 255

y_train = np.load(undersample_path + 'Y9Prop0.1.npy')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [14]:
#Set up the model function
def model_func():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    # initiate RMSprop optimizer
    opt = tensorflow.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
    
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    return model

In [ ]:
model_store = []
history_store = []
alpha_store = []
weight_store = [np.array(x_train.shape[0]*[1/x_train.shape[0]])] #Initialize as 1/n for all
performance_matrix = np.zeros((num_classes,num_iterators)) #Accuracy of nth model on mth class

for i in range(num_iterators):
    #Create current model, train it, store it and it's history.
    curr_model = model_func()
    curr_history = curr_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, sample_weight=weight_store[i], validation_data=(x_test, y_test), shuffle=True)
    model_store.append(curr_model)
    history_store.append(curr_history)
    
    #Recalculate the weights, update weights
    prediction = curr_model.predict(x_train) #predict    
    predict_matrix = (prediction == prediction.max(axis=1)[:,None]).astype(int) #Convert to 1-0 matrix
    match_matrix = y_train - predict_matrix #Difference to eventually generate match_vector
    match_vector = np.sum(np.abs(match_matrix), axis = 1)/2 #1 indicates no match
    error = np.sum(weight_store[i] * match_vector)/np.sum(weight_store[i]) #Calculate error
    alpha = np.log((1-error)/error) + np.log(num_classes-1) #Calulate alpha
    alpha_store.append(alpha) #Append
    new_weight = weight_store[i] * np.exp(alpha * match_vector) #Calculate new weights
    new_weight /= np.sum(new_weight) #Normalize
    weight_store.append(new_weight) #Append
    
    #Update our performance matrix
    prediction_vector = np.argmax(prediction, axis = 1)
    actual_vector = np.argmax(y_train, axis = 1)
    for j in range(len(actual_vector)):
        if actual_vector[j]==prediction_vector[j]:
            performance_matrix[actual_vector[j]][i] += 1
            
    if i%10==0 or i == num_iterators-1:
        np.save('/home/ec2-user/SageMaker/efs/Data/GANBoostStore/Weights/new_weight_re' + str(i) + '.npy', new_weight)
        np.save('/home/ec2-user/SageMaker/efs/Data/GANBoostStore/Alpha/alpha_re' + str(i) + '.npy', alpha_store)
        curr_model.save_weights('/home/ec2-user/SageMaker/efs/Data/GANBoostStore/Model/model_re' + str(i) + '.h5')

Train on 45500 samples, validate on 10000 samples
Epoch 1/30
45500/45500 [==============================] - 10s 211us/sample - loss: 4.8472e-05 - acc: 0.1516 - val_loss: 2.1995 - val_acc: 0.2439
Epoch 2/30
45500/45500 [==============================] - 9s 197us/sample - loss: 4.3400e-05 - acc: 0.2555 - val_loss: 2.0502 - val_acc: 0.2881
Epoch 3/30
45500/45500 [==============================] - 9s 197us/sample - loss: 4.1114e-05 - acc: 0.3103 - val_loss: 1.9284 - val_acc: 0.3469
Epoch 4/30
45500/45500 [==============================] - 9s 197us/sample - loss: 3.8791e-05 - acc: 0.3557 - val_loss: 1.8820 - val_acc: 0.3607
Epoch 5/30
45500/45500 [==============================] - 9s 198us/sample - loss: 3.7216e-05 - acc: 0.3815 - val_loss: 1.7696 - val_acc: 0.3875
Epoch 6/30
45500/45500 [==============================] - 9s 199us/sample - loss: 3.6088e-05 - acc: 0.4031 - val_loss: 1.7716 - val_acc: 0.3833
Epoch 7/30
45500/45500 [==============================] - 9s 199us/sample - loss: 3.5

Epoch 27/30
45500/45500 [==============================] - 9s 200us/sample - loss: 3.9605e-05 - acc: 0.3361 - val_loss: 1.6441 - val_acc: 0.3705
Epoch 28/30
45500/45500 [==============================] - 9s 200us/sample - loss: 3.9514e-05 - acc: 0.3429 - val_loss: 1.6642 - val_acc: 0.3699
Epoch 29/30
45500/45500 [==============================] - 9s 199us/sample - loss: 3.9457e-05 - acc: 0.3437 - val_loss: 1.6532 - val_acc: 0.3657
Epoch 30/30
45500/45500 [==============================] - 9s 199us/sample - loss: 3.9196e-05 - acc: 0.3486 - val_loss: 1.6267 - val_acc: 0.3854
Train on 45500 samples, validate on 10000 samples
Epoch 1/30
45500/45500 [==============================] - 10s 212us/sample - loss: 4.9092e-05 - acc: 0.1150 - val_loss: 2.3431 - val_acc: 0.1012
Epoch 2/30
45500/45500 [==============================] - 9s 199us/sample - loss: 4.8241e-05 - acc: 0.1376 - val_loss: 2.2330 - val_acc: 0.1569
Epoch 3/30
45500/45500 [==============================] - 9s 200us/sample - loss:

Epoch 23/30
45500/45500 [==============================] - 9s 198us/sample - loss: 4.4115e-05 - acc: 0.2938 - val_loss: 1.8558 - val_acc: 0.3351
Epoch 24/30
45500/45500 [==============================] - 9s 198us/sample - loss: 4.4014e-05 - acc: 0.2948 - val_loss: 1.8493 - val_acc: 0.3278
Epoch 25/30
45500/45500 [==============================] - 9s 198us/sample - loss: 4.3798e-05 - acc: 0.3018 - val_loss: 1.8607 - val_acc: 0.3331
Epoch 26/30
45500/45500 [==============================] - 9s 199us/sample - loss: 4.3733e-05 - acc: 0.2976 - val_loss: 1.8299 - val_acc: 0.3313
Epoch 27/30
45500/45500 [==============================] - 9s 198us/sample - loss: 4.3594e-05 - acc: 0.3017 - val_loss: 1.8178 - val_acc: 0.3222
Epoch 28/30
45500/45500 [==============================] - 9s 198us/sample - loss: 4.3443e-05 - acc: 0.3030 - val_loss: 1.8115 - val_acc: 0.3613
Epoch 29/30
45500/45500 [==============================] - 9s 197us/sample - loss: 4.3280e-05 - acc: 0.3103 - val_loss: 1.7871 - v

Epoch 19/30
45500/45500 [==============================] - 9s 201us/sample - loss: 4.6290e-05 - acc: 0.2117 - val_loss: 2.0224 - val_acc: 0.2220
Epoch 20/30
45500/45500 [==============================] - 9s 201us/sample - loss: 4.6125e-05 - acc: 0.2127 - val_loss: 1.9978 - val_acc: 0.2263
Epoch 21/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.6090e-05 - acc: 0.2167 - val_loss: 2.0242 - val_acc: 0.2217
Epoch 22/30
45500/45500 [==============================] - 9s 201us/sample - loss: 4.6023e-05 - acc: 0.2174 - val_loss: 2.0142 - val_acc: 0.2280
Epoch 23/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.5916e-05 - acc: 0.2212 - val_loss: 1.9729 - val_acc: 0.2430
Epoch 24/30
45500/45500 [==============================] - 9s 201us/sample - loss: 4.5851e-05 - acc: 0.2222 - val_loss: 2.0119 - val_acc: 0.2200
Epoch 25/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.5731e-05 - acc: 0.2308 - val_loss: 1.9703 - v

Epoch 15/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.7326e-05 - acc: 0.2034 - val_loss: 2.1059 - val_acc: 0.1912
Epoch 16/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.7217e-05 - acc: 0.2038 - val_loss: 2.0987 - val_acc: 0.2086
Epoch 17/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.7161e-05 - acc: 0.2079 - val_loss: 2.0714 - val_acc: 0.2251
Epoch 18/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.6968e-05 - acc: 0.2139 - val_loss: 2.0767 - val_acc: 0.2189
Epoch 19/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.6992e-05 - acc: 0.2141 - val_loss: 2.0529 - val_acc: 0.2420
Epoch 20/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.6906e-05 - acc: 0.2202 - val_loss: 2.0475 - val_acc: 0.2483
Epoch 21/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.6771e-05 - acc: 0.2206 - val_loss: 2.0297 - v

...

45500/45500 [==============================] - 9s 200us/sample - loss: 4.9532e-05 - acc: 0.1290 - val_loss: 2.2382 - val_acc: 0.1406
Epoch 22/30
45500/45500 [==============================] - 9s 201us/sample - loss: 4.9439e-05 - acc: 0.1362 - val_loss: 2.2375 - val_acc: 0.1400
Epoch 23/30
45500/45500 [==============================] - 9s 202us/sample - loss: 4.9440e-05 - acc: 0.1357 - val_loss: 2.2386 - val_acc: 0.1332
Epoch 24/30
45500/45500 [==============================] - 9s 200us/sample - loss: 4.9399e-05 - acc: 0.1391 - val_loss: 2.2254 - val_acc: 0.1349
Epoch 25/30
45500/45500 [==============================] - 9s 202us/sample - loss: 4.9354e-05 - acc: 0.1452 - val_loss: 2.2163 - val_acc: 0.1430
Epoch 26/30
45500/45500 [==============================] - 9s 201us/sample - loss: 4.9314e-05 - acc: 0.1417 - val_loss: 2.2088 - val_acc: 0.1718
Epoch 27/30
45500/45500 [==============================] - 9s 201us/sample - loss: 4.9273e-05 - acc: 0.1498 - val_loss: 2.2220 - val_acc: 0.13

45500/45500 [==============================] - 9s 201us/sample - loss: 4.9803e-05 - acc: 0.1333 - val_loss: 2.2587 - val_acc: 0.1196
Epoch 18/30
45500/45500 [==============================] - 9s 202us/sample - loss: 4.9794e-05 - acc: 0.1295 - val_loss: 2.2520 - val_acc: 0.1506
Epoch 19/30
45500/45500 [==============================] - 9s 202us/sample - loss: 4.9736e-05 - acc: 0.1397 - val_loss: 2.2455 - val_acc: 0.1402
Epoch 20/30
45500/45500 [==============================] - 9s 203us/sample - loss: 4.9691e-05 - acc: 0.1408 - val_loss: 2.2388 - val_acc: 0.1391
Epoch 21/30
45500/45500 [==============================] - 9s 203us/sample - loss: 4.9643e-05 - acc: 0.1450 - val_loss: 2.2308 - val_acc: 0.1588
Epoch 22/30
45500/45500 [==============================] - 9s 203us/sample - loss: 4.9579e-05 - acc: 0.1417 - val_loss: 2.2252 - val_acc: 0.1921
Epoch 23/30
45500/45500 [==============================] - 9s 202us/sample - loss: 4.9609e-05 - acc: 0.1578 - val_loss: 2.2197 - val_acc: 0.14

45500/45500 [==============================] - 9s 202us/sample - loss: 5.0045e-05 - acc: 0.1373 - val_loss: 2.2731 - val_acc: 0.1878
Epoch 14/30
45500/45500 [==============================] - 9s 203us/sample - loss: 5.0007e-05 - acc: 0.1519 - val_loss: 2.2678 - val_acc: 0.1599
Epoch 15/30
45500/45500 [==============================] - 9s 203us/sample - loss: 4.9988e-05 - acc: 0.1508 - val_loss: 2.2587 - val_acc: 0.1878
Epoch 16/30
45500/45500 [==============================] - 9s 203us/sample - loss: 4.9969e-05 - acc: 0.1541 - val_loss: 2.2578 - val_acc: 0.1879
Epoch 17/30
45500/45500 [==============================] - 9s 201us/sample - loss: 4.9942e-05 - acc: 0.1505 - val_loss: 2.2589 - val_acc: 0.1937
Epoch 18/30
45500/45500 [==============================] - 9s 204us/sample - loss: 4.9920e-05 - acc: 0.1513 - val_loss: 2.2470 - val_acc: 0.1991
Epoch 19/30
45500/45500 [==============================] - 9s 203us/sample - loss: 4.9932e-05 - acc: 0.1583 - val_loss: 2.2477 - val_acc: 0.21

Epoch 9/30
45500/45500 [==============================] - 9s 199us/sample - loss: 5.0188e-05 - acc: 0.1125 - val_loss: 2.2964 - val_acc: 0.1202
Epoch 10/30
45500/45500 [==============================] - 9s 201us/sample - loss: 5.0239e-05 - acc: 0.1137 - val_loss: 2.2968 - val_acc: 0.1124
Epoch 11/30
45500/45500 [==============================] - 9s 200us/sample - loss: 5.0170e-05 - acc: 0.1152 - val_loss: 2.2932 - val_acc: 0.1121
Epoch 12/30
45500/45500 [==============================] - 9s 200us/sample - loss: 5.0198e-05 - acc: 0.1139 - val_loss: 2.2900 - val_acc: 0.1066
Epoch 13/30
45500/45500 [==============================] - 9s 200us/sample - loss: 5.0161e-05 - acc: 0.1176 - val_loss: 2.2914 - val_acc: 0.1179
Epoch 14/30
45500/45500 [==============================] - 9s 202us/sample - loss: 5.0135e-05 - acc: 0.1161 - val_loss: 2.2857 - val_acc: 0.1041
Epoch 15/30
45500/45500 [==============================] - 9s 201us/sample - loss: 5.0109e-05 - acc: 0.1178 - val_loss: 2.2825 - va

Epoch 5/30
45500/45500 [==============================] - 9s 202us/sample - loss: 5.0256e-05 - acc: 0.1093 - val_loss: 2.3148 - val_acc: 0.1070
Epoch 6/30
45500/45500 [==============================] - 9s 202us/sample - loss: 5.0218e-05 - acc: 0.1087 - val_loss: 2.3115 - val_acc: 0.0999
Epoch 7/30
45500/45500 [==============================] - 9s 201us/sample - loss: 5.0190e-05 - acc: 0.1093 - val_loss: 2.3088 - val_acc: 0.1039
Epoch 8/30
45500/45500 [==============================] - 9s 202us/sample - loss: 5.0197e-05 - acc: 0.1084 - val_loss: 2.3031 - val_acc: 0.1017
Epoch 9/30
45500/45500 [==============================] - 9s 202us/sample - loss: 5.0117e-05 - acc: 0.1104 - val_loss: 2.3128 - val_acc: 0.0995
Epoch 10/30
45500/45500 [==============================] - 9s 202us/sample - loss: 5.0161e-05 - acc: 0.1114 - val_loss: 2.3022 - val_acc: 0.0991
Epoch 11/30
45500/45500 [==============================] - 9s 203us/sample - loss: 5.0168e-05 - acc: 0.1095 - val_loss: 2.3020 - val_ac

In [ ]:
#Change performance matrix to accuracy figures:
normed_perf_matrix = np.zeros_like(performance_matrix)

for i in range(10):
    i_count = list(np.argmax(y_train,axis=1)).count(i)
    normed_perf_matrix[i] = performance_matrix[i]/i_count

#Normalize for algorithm:
normed_perf_matrix /= 0.5

In [ ]:
#Make the final prediction
final_matrix = np.ones((len(x_test), num_classes))

for i in range(len(model_store)):
    prediction_vector = np.argmax(model_store[i].predict(x_test), axis = 1)
    for j in range(len(prediction_vector)):
        final_matrix[j][prediction_vector[j]] = final_matrix[j][prediction_vector[j]] * normed_perf_matrix[prediction_vector[j]][i]